In [0]:
import keras
import numpy as np
import pandas as pd
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Flatten,Activation
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, BatchNormalization,GlobalAveragePooling2D
from keras.models import load_model,Model
import time
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, BatchNormalization
from keras import regularizers
from sklearn.metrics import classification_report
from datetime import timedelta
from keras.applications.inception_v3 import InceptionV3, preprocess_input

np.random.seed(666)

Using TensorFlow backend.


In [0]:
batch_size = 128

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        validation_split = 0.2)


train_generator = train_datagen.flow_from_directory(
        'img',
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        'img',
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation')

Found 4000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [0]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(32, 32, 3), kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.00025, decay=1e-6),
                  metrics=['accuracy'])

In [0]:
#Pretrain model on STL-10
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0,restore_best_weights=True)
modelpath="models/complete_model-{epoch:02d}-{val_acc:.4f}-loss-{val_loss:.4f}.h5"
checkpoint = ModelCheckpoint(modelpath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 200,
    verbose=2,
    callbacks=[keras.callbacks.History(), checkpoint, early_stopping])

Epoch 1/200
 - 6s - loss: 2.6027 - acc: 0.2124 - val_loss: 1.9268 - val_acc: 0.3203
Epoch 2/200
 - 6s - loss: 2.0446 - acc: 0.3171 - val_loss: 1.7302 - val_acc: 0.3796
Epoch 3/200
 - 6s - loss: 1.8529 - acc: 0.3702 - val_loss: 1.6921 - val_acc: 0.3830
Epoch 4/200
 - 6s - loss: 1.7580 - acc: 0.3959 - val_loss: 1.6080 - val_acc: 0.4243
Epoch 5/200
 - 5s - loss: 1.6698 - acc: 0.4019 - val_loss: 1.6335 - val_acc: 0.4255
Epoch 6/200
 - 6s - loss: 1.6423 - acc: 0.4221 - val_loss: 1.4868 - val_acc: 0.4656
Epoch 7/200
 - 5s - loss: 1.5269 - acc: 0.4676 - val_loss: 1.5878 - val_acc: 0.4461
Epoch 8/200
 - 5s - loss: 1.4637 - acc: 0.4753 - val_loss: 1.4325 - val_acc: 0.4702
Epoch 9/200
 - 5s - loss: 1.4523 - acc: 0.4766 - val_loss: 1.4390 - val_acc: 0.4933
Epoch 10/200
 - 6s - loss: 1.3993 - acc: 0.4889 - val_loss: 1.5003 - val_acc: 0.4702
Epoch 11/200
 - 5s - loss: 1.3803 - acc: 0.5055 - val_loss: 1.3589 - val_acc: 0.5046
Epoch 12/200
 - 5s - loss: 1.3366 - acc: 0.5265 - val_loss: 1.4747 - val_a

In [0]:
model_path =  'models/base_model_dense-37-0.6422-loss-1.0778.h5'
model.load_weights(model_path)

In [0]:
freeze_steps = [2, 5, 10, 13, 17, 20, 24, 25]
for i,layer in enumerate(model.layers[:]):
    print(i,layer, layer.trainable)

0 <keras.layers.convolutional.Conv2D object at 0x7f9f26423a20> True
1 <keras.layers.core.Activation object at 0x7f9f26423eb8> True
2 <keras.layers.normalization.BatchNormalization object at 0x7f9f2643d208> True
3 <keras.layers.convolutional.Conv2D object at 0x7f9f2643d438> True
4 <keras.layers.core.Activation object at 0x7f9f2643da58> True
5 <keras.layers.normalization.BatchNormalization object at 0x7f9f71653668> True
6 <keras.layers.pooling.MaxPooling2D object at 0x7f9f715c8438> True
7 <keras.layers.core.Dropout object at 0x7f9f715c88d0> True
8 <keras.layers.convolutional.Conv2D object at 0x7f9f715eaac8> True
9 <keras.layers.core.Activation object at 0x7f9f715eab38> True
10 <keras.layers.normalization.BatchNormalization object at 0x7f9f71532f28> True
11 <keras.layers.convolutional.Conv2D object at 0x7f9f714c59b0> True
12 <keras.layers.core.Activation object at 0x7f9f714c5e48> True
13 <keras.layers.pooling.MaxPooling2D object at 0x7f9f714a3eb8> True
14 <keras.layers.core.Dropout object

In [0]:
def new_model():
    weight_decay = 1e-4
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same', input_shape=(32, 32, 3), kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    return model

In [0]:
#Load pretrained model weights and freeze <freeze_count> layers
def load_weights_freeze_given_layers(model, weight_path, freeze_count):
    model.load_weights(weight_path)
    for layer in model.layers[:freeze_count]:
        layer.trainable = False 

    return model
# Get n randomly selected indices from the data 
def getTrainingDataIndicesOfSize(data, size):
    chosen_indices = np.random.choice(np.arange(len(data)), size, replace=False)
    return chosen_indices
# Simple function to get a new sample from the given class
def getNewDataOfClass(original_data, class_id):
    chosen_index = np.random.choice(np.arange(len(original_data)), 1, replace=False)[0]
    while(original_data[chosen_index][0] != class_id):
        chosen_index = np.random.choice(np.arange(len(original_data)), 1, replace=False)[0]
    return chosen_index
# Add new samples of each class until the sample-size-per-class target is reached
def fillLowSampleCountClassesTo(original_data, indices, target_count):
    unique, counts = np.unique(original_data[indices], return_counts=True)
    combined_stats = np.asarray((unique, counts)).T
    for i in range(len(combined_stats)):
        samples_needed = target_count - combined_stats[i][1]
        while(samples_needed > 0):
            potential_index = getNewDataOfClass(original_data, combined_stats[i][0])
            if not (np.any(indices[:] == potential_index)):
                indices = np.append(indices,potential_index)
                samples_needed -= 1
    return indices
# Remove samples of each class until the sample-size-per-class target is reached
def deleteSamplesOverPerClassLimit(original_data, indices, target_count):
    unique, counts = np.unique(original_data[indices], return_counts=True)
    combined_stats = np.asarray((unique, counts)).T
    for i in range(len(combined_stats)):
        samples_to_delete = combined_stats[i][1] - target_count
        while(samples_to_delete > 0):
            potential_index = np.random.choice(np.arange(len(indices)), 1, replace=False)[0]
            if (original_data[indices[potential_index]] == combined_stats[i][0]):
                indices = np.delete(indices,potential_index)
                samples_to_delete -= 1
    return indices
# Get exactly n samples of each class randomly
# 1. step: get n_class times sample_per_class indices (this gives approximately uniform distribution)
# 2-3. step: correct sample-per-class differences by removing excess samples and adding to underrepresented classes
def getRandomSamplesOfEachClassOfSize(data, n_classes, sample_per_class):
    indices = getTrainingDataIndicesOfSize(data, n_classes*sample_per_class)
    indices = fillLowSampleCountClassesTo(data, indices, sample_per_class)
    indices = deleteSamplesOverPerClassLimit(data, indices, sample_per_class)
  
  
    return indices

def classification_report_dataframe(report, index=0):
    report_data = []
    lines = report.split('\n')
    row = {}

    for line in lines[2:-5]:
        row_data = line.split('      ')
        class_idx = str(int(row_data[1]))
        row[class_idx + 'precision'] = float(row_data[2])
        row[class_idx + 'recall'] = float(row_data[3])
        row[class_idx + 'f1_score'] = float(row_data[4])
    
    dataframe = pd.DataFrame(data=row, index=[index])
    
    
    return dataframe

In [0]:
#Experiments on CIFAR-10
freeze_steps =  [0, 2, 5, 10, 13, 17, 20, 23]

sample_counts = [50,100,200,400,700,1000,1500,2000,3000,4000,5000]

n_classes = 10
n_iters = 10

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
start_time = time.time()

weight_path = 'models/complete_model_dense-32-0.6353-loss-1.0800.h5'

for i in range(len(sample_counts)):
    sample_size = sample_counts[i]
    print("--- SAMPLE SIZE:", sample_size, "---")
    for j in range(len(freeze_steps)):
        layers_to_freeze = freeze_steps[j]
        test_scores =  pd.DataFrame()
        print("Frozen layer count: " , layers_to_freeze)

        for iteration in range(n_iters):
                print("\t Iteration: ", iteration)
                model = new_model()
                model = load_weights_freeze_given_layers(model, weight_path, layers_to_freeze)
                model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=0.00025, decay=1e-6),
                      metrics=['accuracy'])
                indices = getRandomSamplesOfEachClassOfSize(Y_train, n_classes, sample_size)
                x_train = X_train[indices]
                y_train = Y_train[indices]
                x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, shuffle= True)
                modelpath="models/" + str(sample_size) + "/" + str(layers_to_freeze) + "/" + str(iteration) + "-base_model.h5"
                checkpoint = ModelCheckpoint(modelpath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)

                train_logpath = "logs/" + "training/" + str(sample_size) + "/" + str(layers_to_freeze) + "/" + str(iteration) + "-base_model.csv"
                train_logger = CSVLogger(train_logpath, append=False, separator=';')

                early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0,restore_best_weights=True)
        
                history = model.fit(x_train / 255.0, to_categorical(y_train),
                          batch_size=128,
                          shuffle=True,
                          epochs=200,
                          validation_data=(x_valid / 255.0, to_categorical(y_valid)),
                          callbacks=[keras.callbacks.History(), checkpoint, train_logger, early_stopping],
                                   verbose = 0)
                score = model.evaluate(X_test / 255.0, to_categorical(Y_test), batch_size=128, verbose=0)
                score = dict(zip(model.metrics_names, score))
                score_df = pd.DataFrame(data=score, index=[0])
                y_pred = model.predict_classes(X_test)
                report = classification_report(Y_test, y_pred)
                report_df = classification_report_dataframe(report)
                test_scores = test_scores.append(pd.concat([score_df, report_df], axis=1))
        test_logpath = "logs/" + "/test/" + str(sample_size) + "/" + str(layers_to_freeze) + "/"+ "base_model.csv"
        test_scores.to_csv(test_logpath)
        print("%.2f%% (+/- %.2f%%)" % (np.mean(test_scores['acc']), np.std(test_scores['acc'])))
end_time = time.time()
time_dif = end_time - start_time
print("Total elapsed time: ",  str(timedelta(seconds=int(round(time_dif)))))

In [0]:
train_data_dir = 'analyzer_calc_data/train'
validation_data_dir = 'analyzer_calc_data/validation'
test_data_dir = 'analyzer_calc_data/test'

nb_train_samples = 2146
nb_validation_samples = 604
nb_test_samples = 604
nb_epoch = 60
pretrain_epoch = 20
batch_size = 12

img_height = 299
img_width = 299
train_datagen = ImageDataGenerator(
rotation_range= 30,
zoom_range = 0.3,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True,
vertical_flip = True,
rescale = 1./255,
fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_width, img_height),
class_mode = 'binary',
batch_size = batch_size,
shuffle = True,
seed = 666)
validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_width, img_height),
class_mode = 'binary',
batch_size = batch_size,
shuffle = True,
seed = 666)
test_generator = test_datagen.flow_from_directory(
test_data_dir,
target_size = (img_width, img_height),
class_mode = 'binary',
batch_size = batch_size,
shuffle = True,
seed = 666)


Found 2146 images belonging to 2 classes.
Found 604 images belonging to 2 classes.
Found 604 images belonging to 2 classes.


In [0]:
#Freeze steps corresponding to Inception modules
freeze_steps =  [41, 64, 87, 101, 133, 165, 197, 229, 249]


n_iters = 4

start_time = time.time()



for j in range(len(freeze_steps)):
    layers_to_freeze = freeze_steps[j]
    test_scores =  pd.DataFrame()
    print("Frozen layer count: " , layers_to_freeze)

    for iteration in range(n_iters):
            print("\t Iteration: ", iteration)
            loaded_model = InceptionV3(include_top=False, weights='imagenet')
            top = loaded_model.output
            top = GlobalAveragePooling2D()(top)
            top = Dense(1248, activation='relu')(top)
            top = Dropout(0.4)(top)


            classifier = Dense(1, activation='sigmoid')(top)
            model = Model(inputs=loaded_model.input, outputs=classifier)
            for layer in model.layers[:layers_to_freeze]:
                layer.trainable = False
            adam = Adam(lr=0.00035, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            model.compile(loss='binary_crossentropy',
                          optimizer= adam,
                          metrics=['accuracy'])
            
            modelpath="calc_models/" + str(layers_to_freeze) + "/" + str(iteration) + "-inception_model.h5"
            checkpoint = ModelCheckpoint(modelpath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)

            train_logpath = "calc_logs/" + "training/"  + str(layers_to_freeze) + "/" + str(iteration) + "-inception_model.csv"
            train_logger = CSVLogger(train_logpath, append=False, separator=';')

            early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0,restore_best_weights=True)
            
            history = model.fit_generator(
                train_generator,
                steps_per_epoch = nb_train_samples / batch_size,
                epochs =2, #epochs =nb_epoch,
                verbose = 2,
                validation_data = validation_generator,
                validation_steps = nb_validation_samples /batch_size,
                callbacks=[keras.callbacks.History(), checkpoint, train_logger, early_stopping])

            score = model.evaluate_generator(test_generator, nb_test_samples)
            score = dict(zip(model.metrics_names, score))
            print(score)
            score_df = pd.DataFrame(data=score, index=[0])

            test_scores = test_scores.append(score_df)
    test_logpath = "calc_logs/" + "test" + "/" + str(layers_to_freeze) + "/"+ "inception_model.csv"
    test_scores.to_csv(test_logpath)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(test_scores['acc']), np.std(test_scores['acc'])))
end_time = time.time()
time_dif = end_time - start_time
print("Total elapsed time: ",  str(timedelta(seconds=int(round(time_dif)))))